# Imports

In [1]:
import io
import sys
import os

import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd 
%matplotlib inline

In [2]:
from pathlib import Path
import json
import subprocess

In [3]:
import boto3
import sagemaker
import docker
from sagemaker import get_execution_role

In [4]:
from sagemaker.sklearn import SKLearn
from sagemaker.local import LocalSession

In [5]:
from sklearn.model_selection import train_test_split

# Set up session

In [6]:
# sagemaker session, role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# S3 bucket name
bucket = sagemaker_session.default_bucket()

In [28]:
ENDPOINT_NAME = "credit-explainer"

In [8]:
AWS_ACCOUNT_ID = role.split(":")[4]
AWS_REGION = sagemaker_session.boto_region_name

## ECR Repository

Given that we will be deploying a custom docker container, need to make sure we have a ECR docker registry set up.

You can go the the console to create one (mine is in `eu-central-1`, so go to https://eu-central-1.console.aws.amazon.com/ecr/repositories?region=eu-central-1)

The name I gave to my repository is `sagemaker-explainer`.

In [9]:
ECR_REPOSITORY_NAME = 'sagemaker-explainer'
CUSTOM_IMAGE_URI = str(AWS_ACCOUNT_ID)+'.dkr.ecr.' + AWS_REGION + '.amazonaws.com/' + ECR_REPOSITORY_NAME

# Load data

In [10]:
data_dir = Path.cwd().parent / "data"
data_dir

PosixPath('/home/ec2-user/SageMaker/sagemaker-creditscore-explainer/sagemaker/data')

In [11]:
train_df = pd.read_csv(data_dir / "train.csv")
test_df = pd.read_csv(data_dir / "test.csv")

In [12]:
TARGET = 'SeriousDlqin2yrs'

In [13]:
train_df

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,0,1.000000,29,0,0.167191,2541.0,3,0,0,0,2.0
1,0,0.022649,65,0,0.402510,9241.0,12,0,2,0,1.0
2,0,0.435809,55,1,0.380973,6800.0,25,1,1,1,2.0
3,0,0.056438,56,0,1740.000000,NaN,5,0,1,0,0.0
4,0,0.254972,29,2,0.013997,5000.0,4,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
17995,0,0.000000,79,0,0.000000,NaN,4,0,0,0,NaN
17996,0,0.008116,64,0,0.117417,10500.0,9,0,1,0,0.0
17997,0,0.141855,76,1,0.206648,2917.0,9,0,0,0,0.0
17998,0,0.831707,64,0,2782.000000,NaN,18,0,0,0,0.0


In [14]:
# set prefix, a descriptive name for a directory  
bucket_prefix = 'sagemaker-explainer'
model_output_prefix = 'model_output'

# upload all data to S3
train_data = sagemaker_session.upload_data(str(data_dir / "train.csv"), bucket, key_prefix=bucket_prefix)
test_data = sagemaker_session.upload_data(str(data_dir / "test.csv"), bucket, key_prefix=bucket_prefix)

# Set up custom docker training container

## install `docker-credential-ecr-login`

Before we can deploy a custom docker image that includes the shap library, first have to install `docker-credential-ecr-login`

Uncomment the cells below install `docker-credential-ecr-login`:

In [15]:
# uncomment this line below in order to install amazon-ecr-credential-helper:
!sudo wget -P /usr/bin https://amazon-ecr-credential-helper-releases.s3.us-east-2.amazonaws.com/0.4.0/linux-amd64/docker-credential-ecr-login
              

--2020-08-17 18:05:30--  https://amazon-ecr-credential-helper-releases.s3.us-east-2.amazonaws.com/0.4.0/linux-amd64/docker-credential-ecr-login
Resolving amazon-ecr-credential-helper-releases.s3.us-east-2.amazonaws.com (amazon-ecr-credential-helper-releases.s3.us-east-2.amazonaws.com)... 52.219.100.176
Connecting to amazon-ecr-credential-helper-releases.s3.us-east-2.amazonaws.com (amazon-ecr-credential-helper-releases.s3.us-east-2.amazonaws.com)|52.219.100.176|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8577024 (8.2M) [binary/octet-stream]
Saving to: ‘/usr/bin/docker-credential-ecr-login’

docker-credential-e 100%[===================>]   8.18M  5.93MB/s    in 1.4s    

2020-08-17 18:05:32 (5.93 MB/s) - ‘/usr/bin/docker-credential-ecr-login’ saved [8577024/8577024]



In [16]:
# uncomment this line below in order to change permission on /usr/bin/docker-credential-ecr-login:
!sudo chmod +x /usr/bin/docker-credential-ecr-login

## Attach `AmazonEC2ContainerRegistryFullAccess` policy to sagemaker notebook

In order to create a custom docker container containing the shap library for our model we need to add `AmazonEC2ContainerRegistryFullAccess` policy to our notebook.

In the sagemaker console:
    - click on notebook instances
    - click on the notebook instance that you are using
    - go to Permissions and encryption
    - click on the `IAM role ARN`
    - click on 'Attach Policies'
    - find `AmazonEC2ContainerRegistryFullAccess`
    - add it to the notebook.

### Attach additional policies: 


You may have to add some additional permissions to your notebook policies, namely `"ecr:GetDownloadUrlForLayer"`, `"ecr:BatchGetImage"` and `"ecr:BatchCheckLayerAvailability"`. 

You can either edit these manually, or paste the following json:

```json
{
    "Version": "2008-10-17",
    "Statement": [
        {
            "Sid": "allowSageMakerToPull",
            "Effect": "Allow",
            "Action": [
                "ecr:GetDownloadUrlForLayer",
                "ecr:BatchGetImage",
                "ecr:BatchCheckLayerAvailability"
            ],
            "Resource": "*"
        }
    ]
}
```

In [17]:
ecr_client = boto3.client("ecr", region_name=AWS_REGION)
docker_client = docker.APIClient()

class DockerImage:
    def __init__(self, registry, repository_name, tag="latest",
                docker_config_filepath='/home/ec2-user/.docker/config.json'):
        self.registry = registry
        self.repository_name = repository_name
        self.docker_config_filepath = docker_config_filepath
        self.tag = tag
        self._check_credential_manager()
        self._configure_credentials()

    def __str__(self):
        return "{}/{}:{}".format(self.registry, self.repository_name, self.tag)

    @property
    def repository(self):
        return "{}/{}".format(self.registry, self.repository_name)

    @property
    def short_name(self):
        return self.repository_name

    @staticmethod
    def _check_credential_manager():
        try:
            subprocess.run(
                ["docker-credential-ecr-login", "version"],
                stdout=subprocess.DEVNULL,
            )
        except Exception:
            raise Exception(
                "Couldn't run 'docker-credential-ecr-login'. "
                "Make sure it is installed and configured correctly."
            )

    def _configure_credentials(self):
        docker_config_filepath = Path(self.docker_config_filepath)
        if docker_config_filepath.exists():
            with open(docker_config_filepath, "r") as openfile:
                docker_config = json.load(openfile)
        else:
            docker_config = {}
        if "credHelpers" not in docker_config:
            docker_config["credHelpers"] = {}
        docker_config["credHelpers"][self.registry] = "ecr-login"
        docker_config_filepath.parent.mkdir(exist_ok=True, parents=True)
        with open(docker_config_filepath, "w") as openfile:
            json.dump(docker_config, openfile, indent=4)

    def build(self, dockerfile, buildargs):
        path = Path(dockerfile).parent
        for line in docker_client.build(
            path=str(path),
            buildargs=buildargs,
            tag=self.repository_name,
            decode=True,
        ):
            if "error" in line:
                raise Exception(line["error"])
            else:
                print(line)

    def push(self):
        docker_client.tag(
            self.repository_name, self.repository, self.tag, force=True
        )
        for line in docker_client.push(
            self.repository, self.tag, stream=True, decode=True
        ):
            print(line)

## Get base scikit-learn image

This is the base image for our docker image.

In [18]:
def scikit_learn_image():
    registry = sagemaker.fw_registry.registry(
        region_name=AWS_REGION, framework="scikit-learn"
    )
    repository_name = "sagemaker-scikit-learn"
    tag = "0.20.0-cpu-py3"
    return DockerImage(registry, repository_name, tag)

sklearn_image = scikit_learn_image()
str(sklearn_image)

'492215442770.dkr.ecr.eu-central-1.amazonaws.com/sagemaker-scikit-learn:0.20.0-cpu-py3'

## Build custom image based on this scikit-learn image

The `Dockerfile` looks as follows:

```Dockerfile
ARG SCIKIT_LEARN_IMAGE
FROM $SCIKIT_LEARN_IMAGE

COPY requirements.txt /requirements.txt
RUN pip install --no-cache -r /requirements.txt && \
    rm /requirements.txt
```

So we start with a `$SCIKIT_LEARN_IMAGE` as a base (given as a parameter), and then install the additional requirements. (in our case `joblib` and `shap`)

In [20]:
def custom_image(aws_account_id, aws_region, repository_name, tag="latest"):
    ecr_registry = f"{aws_account_id}.dkr.ecr.{aws_region}.amazonaws.com"
    return DockerImage(ecr_registry, repository_name, tag)

custom_image = custom_image(AWS_ACCOUNT_ID, AWS_REGION, ECR_REPOSITORY_NAME)

dockerfile = Path.cwd().parent / "container" / "Dockerfile"

custom_image.build(
    dockerfile=dockerfile,
    buildargs={'SCIKIT_LEARN_IMAGE': str(sklearn_image)}
)

custom_image.push()


{'stream': 'Step 1/4 : ARG SCIKIT_LEARN_IMAGE'}
{'stream': '\n'}
{'stream': 'Step 2/4 : FROM $SCIKIT_LEARN_IMAGE'}
{'stream': '\n'}
{'status': 'Pulling from sagemaker-scikit-learn', 'id': '0.20.0-cpu-py3'}
{'status': 'Pulling fs layer', 'progressDetail': {}, 'id': 'dadc679dc38f'}
{'status': 'Pulling fs layer', 'progressDetail': {}, 'id': '9e622765e19e'}
{'status': 'Pulling fs layer', 'progressDetail': {}, 'id': '3742832a87c6'}
{'status': 'Pulling fs layer', 'progressDetail': {}, 'id': '34c6cdf813d1'}
{'status': 'Pulling fs layer', 'progressDetail': {}, 'id': '7582aa0b1ed5'}
{'status': 'Pulling fs layer', 'progressDetail': {}, 'id': 'f42eda49fdd2'}
{'status': 'Pulling fs layer', 'progressDetail': {}, 'id': 'ac8c022aaad2'}
{'status': 'Pulling fs layer', 'progressDetail': {}, 'id': '91cf5329f2d2'}
{'status': 'Pulling fs layer', 'progressDetail': {}, 'id': 'a170f2e57d1e'}
{'status': 'Pulling fs layer', 'progressDetail': {}, 'id': 'e54f9d9f1590'}
{'status': 'Waiting', 'progressDetail': {}, 

{'status': 'Downloading', 'progressDetail': {'current': 56648258, 'total': 143601687}, 'progress': '[===================>                               ]  56.65MB/143.6MB', 'id': '7582aa0b1ed5'}
{'status': 'Extracting', 'progressDetail': {'current': 3670016, 'total': 43250279}, 'progress': '[====>                                              ]   3.67MB/43.25MB', 'id': 'dadc679dc38f'}
{'status': 'Downloading', 'progressDetail': {'current': 10791423, 'total': 378859596}, 'progress': '[=>                                                 ]  10.79MB/378.9MB', 'id': 'ac8c022aaad2'}
{'status': 'Downloading', 'progressDetail': {'current': 49106431, 'total': 143422943}, 'progress': '[=================>                                 ]  49.11MB/143.4MB', 'id': 'f42eda49fdd2'}
{'status': 'Downloading', 'progressDetail': {'current': 62044738, 'total': 143601687}, 'progress': '[=====================>                             ]  62.04MB/143.6MB', 'id': '7582aa0b1ed5'}
{'status': 'Downloading', 'p

{'status': 'Downloading', 'progressDetail': {'current': 46408191, 'total': 378859596}, 'progress': '[======>                                            ]  46.41MB/378.9MB', 'id': 'ac8c022aaad2'}
{'status': 'Extracting', 'progressDetail': {'current': 27983872, 'total': 43250279}, 'progress': '[================================>                  ]  27.98MB/43.25MB', 'id': 'dadc679dc38f'}
{'status': 'Downloading', 'progressDetail': {'current': 124104258, 'total': 143601687}, 'progress': '[===========================================>       ]  124.1MB/143.6MB', 'id': '7582aa0b1ed5'}
{'status': 'Downloading', 'progressDetail': {'current': 79326719, 'total': 143422943}, 'progress': '[===========================>                       ]  79.33MB/143.4MB', 'id': 'f42eda49fdd2'}
{'status': 'Downloading', 'progressDetail': {'current': 48566783, 'total': 378859596}, 'progress': '[======>                                            ]  48.57MB/378.9MB', 'id': 'ac8c022aaad2'}
{'status': 'Extracting', '

{'status': 'Downloading', 'progressDetail': {'current': 131672575, 'total': 143422943}, 'progress': '[=============================================>     ]  131.7MB/143.4MB', 'id': 'f42eda49fdd2'}
{'status': 'Extracting', 'progressDetail': {'current': 42663936, 'total': 43250279}, 'progress': '[=================================================> ]  42.66MB/43.25MB', 'id': 'dadc679dc38f'}
{'status': 'Downloading', 'progressDetail': {'current': 138148351, 'total': 143422943}, 'progress': '[================================================>  ]  138.1MB/143.4MB', 'id': 'f42eda49fdd2'}
{'status': 'Downloading', 'progressDetail': {'current': 21584385, 'total': 56934490}, 'progress': '[==================>                                ]  21.58MB/56.93MB', 'id': '91cf5329f2d2'}
{'status': 'Downloading', 'progressDetail': {'current': 86881791, 'total': 378859596}, 'progress': '[===========>                                       ]  86.88MB/378.9MB', 'id': 'ac8c022aaad2'}
{'status': 'Verifying Chec

{'status': 'Downloading', 'progressDetail': {'current': 246617599, 'total': 378859596}, 'progress': '[================================>                  ]  246.6MB/378.9MB', 'id': 'ac8c022aaad2'}
{'status': 'Downloading', 'progressDetail': {'current': 259029503, 'total': 378859596}, 'progress': '[==================================>                ]    259MB/378.9MB', 'id': 'ac8c022aaad2'}
{'status': 'Downloading', 'progressDetail': {'current': 271441407, 'total': 378859596}, 'progress': '[===================================>               ]  271.4MB/378.9MB', 'id': 'ac8c022aaad2'}
{'status': 'Downloading', 'progressDetail': {'current': 283313663, 'total': 378859596}, 'progress': '[=====================================>             ]  283.3MB/378.9MB', 'id': 'ac8c022aaad2'}
{'status': 'Pull complete', 'progressDetail': {}, 'id': 'dadc679dc38f'}
{'status': 'Downloading', 'progressDetail': {'current': 295725567, 'total': 378859596}, 'progress': '[=======================================>  

...

{'status': 'Pushing', 'progressDetail': {'current': 629592064, 'total': 1076667337}, 'progress': '[=============================>                     ]  629.6MB/1.077GB', 'id': '8be6c85c96e4'}
{'status': 'Pushing', 'progressDetail': {'current': 631257088, 'total': 1076667337}, 'progress': '[=============================>                     ]  631.3MB/1.077GB', 'id': '8be6c85c96e4'}
{'status': 'Pushing', 'progressDetail': {'current': 633485312, 'total': 1076667337}, 'progress': '[=============================>                     ]  633.5MB/1.077GB', 'id': '8be6c85c96e4'}
{'status': 'Pushing', 'progressDetail': {'current': 635156480, 'total': 1076667337}, 'progress': '[=============================>                     ]  635.2MB/1.077GB', 'id': '8be6c85c96e4'}
{'status': 'Pushing', 'progressDetail': {'current': 636827648, 'total': 1076667337}, 'progress': '[=============================>                     ]  636.8MB/1.077GB', 'id': '8be6c85c96e4'}
{'status': 'Pushing', 'progressDeta

{'status': 'Pushing', 'progressDetail': {'current': 730757120, 'total': 1076667337}, 'progress': '[=================================>                 ]  730.8MB/1.077GB', 'id': '8be6c85c96e4'}
{'status': 'Pushing', 'progressDetail': {'current': 733542400, 'total': 1076667337}, 'progress': '[==================================>                ]  733.5MB/1.077GB', 'id': '8be6c85c96e4'}
{'status': 'Pushing', 'progressDetail': {'current': 736327680, 'total': 1076667337}, 'progress': '[==================================>                ]  736.3MB/1.077GB', 'id': '8be6c85c96e4'}
{'status': 'Pushing', 'progressDetail': {'current': 739112960, 'total': 1076667337}, 'progress': '[==================================>                ]  739.1MB/1.077GB', 'id': '8be6c85c96e4'}
{'status': 'Pushing', 'progressDetail': {'current': 741341184, 'total': 1076667337}, 'progress': '[==================================>                ]  741.3MB/1.077GB', 'id': '8be6c85c96e4'}
{'status': 'Pushing', 'progressDeta

{'status': 'Pushing', 'progressDetail': {'current': 888069120, 'total': 1076667337}, 'progress': '[=========================================>         ]  888.1MB/1.077GB', 'id': '8be6c85c96e4'}
{'status': 'Pushing', 'progressDetail': {'current': 890297344, 'total': 1076667337}, 'progress': '[=========================================>         ]  890.3MB/1.077GB', 'id': '8be6c85c96e4'}
{'status': 'Pushing', 'progressDetail': {'current': 892525568, 'total': 1076667337}, 'progress': '[=========================================>         ]  892.5MB/1.077GB', 'id': '8be6c85c96e4'}
{'status': 'Pushing', 'progressDetail': {'current': 894753792, 'total': 1076667337}, 'progress': '[=========================================>         ]  894.8MB/1.077GB', 'id': '8be6c85c96e4'}
{'status': 'Pushing', 'progressDetail': {'current': 896982016, 'total': 1076667337}, 'progress': '[=========================================>         ]    897MB/1.077GB', 'id': '8be6c85c96e4'}
{'status': 'Pushing', 'progressDeta

{'status': 'Pushing', 'progressDetail': {'current': 1002715648, 'total': 1076667337}, 'progress': '[==============================================>    ]  1.003GB/1.077GB', 'id': '8be6c85c96e4'}
{'status': 'Pushing', 'progressDetail': {'current': 1004929024, 'total': 1076667337}, 'progress': '[==============================================>    ]  1.005GB/1.077GB', 'id': '8be6c85c96e4'}
{'status': 'Pushing', 'progressDetail': {'current': 1007120896, 'total': 1076667337}, 'progress': '[==============================================>    ]  1.007GB/1.077GB', 'id': '8be6c85c96e4'}
{'status': 'Pushing', 'progressDetail': {'current': 1009906176, 'total': 1076667337}, 'progress': '[==============================================>    ]   1.01GB/1.077GB', 'id': '8be6c85c96e4'}
{'status': 'Pushing', 'progressDetail': {'current': 1012642816, 'total': 1076667337}, 'progress': '[===============================================>   ]  1.013GB/1.077GB', 'id': '8be6c85c96e4'}
{'status': 'Pushing', 'progres

{'status': 'Pushing', 'progressDetail': {'current': 1113670656, 'total': 1076667337}, 'progress': '[==================================================>]  1.114GB', 'id': '8be6c85c96e4'}
{'status': 'Pushing', 'progressDetail': {'current': 1115769856, 'total': 1076667337}, 'progress': '[==================================================>]  1.116GB', 'id': '8be6c85c96e4'}
{'status': 'Pushing', 'progressDetail': {'current': 1115911680, 'total': 1076667337}, 'progress': '[==================================================>]  1.116GB', 'id': '8be6c85c96e4'}
{'status': 'Pushed', 'progressDetail': {}, 'id': '8be6c85c96e4'}
{'status': 'latest: digest: sha256:d76564711561faf35c653075fcb5867d7fa8a769bbde3d729b4d672214191e33 size: 2841'}
{'progressDetail': {}, 'aux': {'Tag': 'latest', 'Digest': 'sha256:d76564711561faf35c653075fcb5867d7fa8a769bbde3d729b4d672214191e33', 'Size': 2841}}


# Train model

In [21]:
source_dir = Path.cwd().parent / "src"
source_dir

PosixPath('/home/ec2-user/SageMaker/sagemaker-creditscore-explainer/sagemaker/src')

In [22]:
output_path = 's3://' + str(Path(bucket, bucket_prefix, model_output_prefix))
output_path

's3://sagemaker-eu-central-1-360983820924/sagemaker-explainer/model_output'

In [25]:
hyperparameters = {
    # not optimizing hyperparameters for now, not the focus of this project
}

estimator = SKLearn(
    image_name=str(custom_image),
    entry_point='entry_point.py',
    source_dir=str(source_dir),
    hyperparameters=hyperparameters,
    role=role,
    train_instance_count=1,
    train_instance_type='ml.m5.2xlarge', 
    output_path=output_path,
    code_location=output_path,
)

Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.
This is not the latest supported version. If you would like to use version 0.23-1, please add framework_version=0.23-1 to your constructor.


In [26]:
estimator.fit({'train_data': train_data, 'test_data': test_data})

's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


2020-08-17 18:14:57 Starting - Starting the training job...
2020-08-17 18:14:59 Starting - Launching requested ML instances......
2020-08-17 18:16:01 Starting - Preparing the instances for training...
2020-08-17 18:16:36 Downloading - Downloading input data
2020-08-17 18:16:36 Training - Downloading the training image.....2020-08-17 18:17:32,824 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2020-08-17 18:17:32,826 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-08-17 18:17:32,835 sagemaker_sklearn_container.training INFO     Invoking user training script.
2020-08-17 18:17:33,056 sagemaker-containers INFO     Module entry_point does not provide a setup.py. 
Generating setup.py
2020-08-17 18:17:33,056 sagemaker-containers INFO     Generating setup.cfg
2020-08-17 18:17:33,056 sagemaker-containers INFO     Generating MANIFEST.in
2020-08-17 18:17:33,056 sagemaker-containers INFO     Installing module with the followin

# Deploy estimator

In [29]:
ENDPOINT_NAME

'credit-explainer'

In [30]:
estimator.deploy(
    endpoint_name=ENDPOINT_NAME,
    initial_instance_count=1, 
    instance_type="ml.t2.medium")

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


-----------------!

# Create predictor

In [31]:
from sagemaker.predictor import RealTimePredictor
from sagemaker.predictor import json_serializer, json_deserializer, CONTENT_TYPE_JSON

predictor = RealTimePredictor(
    endpoint=endpoint_name,
    sagemaker_session=sagemaker_session,
    serializer=json_serializer,
    deserializer=json_deserializer,
    content_type="application/json",
)

# Test predictor

In [32]:
predictor.predict(test_df.sample(1).to_json(orient='records'))

{'prediction': [0.05187674979012408],
 'shap_base': 0.0606733333333333,
 'shap_values': {'MonthlyIncome': [-0.001911406670069957],
  'age': [0.007089596509830959],
  'NumberOfDependents': [-0.0010889384741045176],
  'DebtRatio': [-0.0007444030564928647],
  'RevolvingUtilizationOfUnsecuredLines': [-0.01696979592690931],
  'NumberRealEstateLoansOrLines': [-0.0006599644194419679],
  'NumberOfOpenCreditLinesAndLoans': [-0.0010915706111867057],
  'NumberOfTime30-59DaysPastDueNotWorse': [0.022754292981416072],
  'NumberOfTime60-89DaysPastDueNotWorse': [-0.004928530423758262],
  'NumberOfTimes90DaysLate': [-0.011245863452492741]}}

# Now build a lambda frontend and GatewayAPI 

- this is a good tutorial: https://medium.com/analytics-vidhya/invoke-an-amazon-sagemaker-endpoint-using-aws-lambda-83ff1a9f5443




## `lambda_handler` code

I used the following code for the lambda handler. I basically just forwards the event and sets ContentTyoe to 'application/json', and decodes the response.

Make sure to define ENDPOINT_NAME in the environmental variables of your lambda function

```python
import os
import io
import boto3
import json
import csv

# grab environment variables
ENDPOINT_NAME = os.environ['ENDPOINT_NAME']
runtime= boto3.client('runtime.sagemaker')

def lambda_handler(event, context):
    print("Received event: " + json.dumps(event, indent=2))
    response = runtime.invoke_endpoint(EndpointName=ENDPOINT_NAME,
                                       ContentType='application/json',
                                       Body=event)
    print("raw response: ", response)
    result = json.loads(response['Body'].read().decode())
    print("decoded result: ", result)
    return result
```